In [1]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
from sklearn import preprocessing
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [2]:
# Data directory
# datadir = '/home/justin/Data/seam/'
datadir = '/mnt/c/Users/Justin/Dropbox (Personal)/Shared/Haneet/Data/'
d_train_f = datadir + 'training_data.mat'
d_test_f = datadir + 'blind_data.mat'

In [3]:
# Load the matlab files for training and testing (blind) boreholes
d_train = io.loadmat(d_train_f)
d_test = io.loadmat(d_test_f)

In [4]:
# Parse the matlab data structures to get training/testing images and labels
labels_train,images_train = d_train['ctrain'][0],d_train['dtrain'][0]
labels_test,images_test = d_test['cblind'][0],d_test['dblind'][0]


In [5]:
# Print some information
print(' ### INFO ###')
print('%i Training Data Boreholes' %(len(images_train)))
print('%i Testing Data Boreholes' %(len(images_test)))
print('%i Classes' %(labels_train[0].shape[0]))
print('%i Values per Image (%i x %i)\n'%(images_train[0].shape[0], np.sqrt(images_train[0].shape[0]), np.sqrt(images_train[0].shape[0])))

cnt = 1
for x,l in zip(images_train,labels_train):
    xv = np.reshape(x,x.size)
    print('Borehole %2i:  %i Depth Slices | Min: %-4.3f   Max: %-4.3f   Mean: % 7.2f   Std: %-4.3f' %(cnt, x.shape[1], xv.min(), xv.max(), xv.mean(), xv. std()))
    cnt += 1

 ### INFO ###
11 Training Data Boreholes
5 Testing Data Boreholes
4 Classes
25 Values per Image (5 x 5)

Borehole  1:  707 Depth Slices | Min: -19841.348   Max: 28279.582   Mean:  -10.96   Std: 4471.869
Borehole  2:  689 Depth Slices | Min: -18987.617   Max: 27663.824   Mean:  -39.07   Std: 4270.490
Borehole  3:  697 Depth Slices | Min: -14817.008   Max: 23764.441   Mean:   62.60   Std: 3502.549
Borehole  4:  691 Depth Slices | Min: -17115.027   Max: 19057.191   Mean:   19.58   Std: 3449.588
Borehole  5:  660 Depth Slices | Min: -18968.973   Max: 21860.555   Mean:    5.75   Std: 3506.990
Borehole  6:  676 Depth Slices | Min: -18961.461   Max: 29092.922   Mean:  -12.60   Std: 4207.515
Borehole  7:  702 Depth Slices | Min: -21064.227   Max: 30714.266   Mean:  -25.05   Std: 4467.483
Borehole  8:  706 Depth Slices | Min: -19675.383   Max: 22419.102   Mean:   24.54   Std: 3528.093
Borehole  9:  583 Depth Slices | Min: -12698.621   Max: 17069.480   Mean:  -47.61   Std: 3405.394
Borehole 10: 

In [ ]:
# Define a plotting function
def montageArray(x, img_size, ncol=50):
    xx = np.reshape(x, (img_size,img_size,-1))
    fig = plt.figure()
    fig.set_size_inches(30,20)
    fig.subplots_adjust(wspace=0.0)
    n = xx.shape[2]
    for i in range(n):   
        a = fig.add_subplot(np.ceil(n/float(ncol)), ncol, i+1)
        
        plt.imshow(xx[:,:,i])
        plt.axis('off')
    plt.show()
        

In [ ]:
# View the data
montageArray(images_train[0],5)

In [6]:
# Change the labels to integers
alllabels = []
for label in labels_train:
    l = [np.where(y)[0][0] for y in label.transpose()]
    alllabels.append(torch.tensor(np.asarray(l)))
    
testlabels = []
for label in labels_test:
    l = [np.where(y)[0][0] for y in label.transpose()]
    testlabels.append(torch.tensor(np.asarray(l)))
    
# print(alllabels)    

In [ ]:
# View the labels
fig = plt.figure()
for i,ll in enumerate(labels):
    a = fig.add_subplot(1, len(labels), i+1)
    plt.plot(100*ll,np.arange(len(ll)),'r.')
    plt.axis('image')
plt.show()

In [7]:
# Normalize the data
alldata = np.concatenate([x.flatten() for x in images_train])
m,s,ma,mi = alldata.mean(),alldata.std(),alldata.max(),alldata.min()
print(m,s,ma,mi)

newd = 2*(alldata - mi)/(ma-mi)-1
print(newd.max(),newd.min(),newd.mean(),newd.std())

Ynorm = [2*(y-mi)/(ma-mi)-1 for y in images_train]
Ynormtest = [2*(y-mi)/(ma-mi)-1 for y in images_test]

-8.227577091130255 3867.0146123165696 30714.265625 -22542.3203125
1.0 -1.0 -0.15375376251665604 0.1452220244404986


In [8]:
# Format the data and labels for PyTorch
# Images should be formatted as [1,1,X,Y,Z] for each Borehole
Ytrain = []
for y in Ynorm:
    Ytrain.append(torch.tensor(np.float32(np.reshape(y,(1,1,5,5,-1)))))
    
Ytest = []
for y in Ynormtest:
    Ytest.append(torch.tensor(np.float32(np.reshape(y,(1,1,5,5,-1)))))

In [9]:
def conv3x3x3(x,K):
    """3x3 convolution with padding"""
    return F.conv3d(x, K, padding=1)

def conv3x3x3T(x,K):
    """3x3 convolution transpose with padding"""
    #K = torch.transpose(K,0,1)
    return F.conv_transpose3d(x, K, padding=1)
        
        
dis = nn.CrossEntropyLoss()
def misfit(X,W,C):    
    n = W.shape
    X = X.view(-1,n[0])
    S = torch.matmul(X,W)
    return dis(S,C), S   

def getAccuracy(S,labels):
    _, predicted = torch.max(S.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct/total

In [10]:
class ResNet(nn.Module):

    def __init__(self, h,NG):
        super().__init__()

        # network geometry
        self.NG       = NG
        # time step
        self.h        = h
        # coarsening and TV norm
        
        
    def forward(self,x,Kresnet):
    
        nt = len(Kresnet)
        
        # time stepping
        for j in range(nt):
            
            # First case - rsent style step
            if NG[0,j] == NG[1,j]: 
                #print(torch.norm(z))
                z  = conv3x3x3(x, Kresnet[j])
                z  = F.instance_norm(z)
                z  = F.relu(z)        
                z  = conv3x3x3T(z,Kresnet[j])
                x  = x - self.h*z
            # Change number of channels/resolution    
            else:
                z  = conv3x3x3(x, Kresnet[j])
                z  = F.instance_norm(z)
                x  = F.relu(z)
        
        # compress in x-y dimensions 
        #x = F.avg_pool3d(x, (5,5,1), stride=None, padding=0)
             
        return x #torch.transpose(p,0,1)
        

In [11]:
# Set up the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [22]:
# initialize net and weights
h           = 1e0

# Network geometry
NG = [1,    16,     16,    16,  
      16,    16,     16,    16, 
      0,    0,     0,    0]

NG = np.reshape(NG,(4,-1))


net   = ResNet(h,NG)

nsteps = NG.shape[1]


Kresnet = []
for i in range(nsteps):  
    Ki  = nn.Parameter(torch.Tensor(np.asscalar(NG[1,i]), np.asscalar(NG[0,i]),3,3,3))
    stdv  = 1e-3
    Ki.data.uniform_(-stdv, stdv)    
    # Move to the GPU
    Ki.data = Ki.data.to(device)
    
    #print(torch.norm(Ki))
    Kresnet.append(Ki)
    
# weights for linear classifier    
W     = nn.Parameter(torch.Tensor(np.asscalar(NG[1,-1])*25,4))
stdv  = 1e-3
W.data.uniform_(-stdv, stdv)
    
# Move to GPU
net.to(device)
W.data = W.data.to(device)

In [23]:
import torch.optim as optim
optimizer = optim.SGD([{'params':Kresnet},{'params': W}], lr=5e-5, momentum=0.9)

# Print every _ iterations
p_iter = 1

# Run _ epochs
n_epoch = 10

In [24]:
for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    running_accuracy = 0.0
    
    print('Epoch   Iteration   Loss(run)   Acc(run)   Acc(val)')
    print('---------------------------------------------------')
    
    for i in range(len(Ytrain)):
        # get the inputs
        inputs = Ytrain[i] 
        labels = alllabels[i]
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        x    = net(inputs,Kresnet)
        loss, Si = misfit(x,W,labels)
        loss.backward()
             
        optimizer.step()

        # print statistics
        accuracy = getAccuracy(Si,labels)
        running_loss     += loss.item()
        running_accuracy += accuracy
        if i % p_iter == (p_iter-1):    # print every p_iter mini-batches
            # compute validation accuracy
            with torch.no_grad():
                for i in range(len(Ytest)):            
                    #    dataiter = iter(testloader)
                    #    inputsV, labelsV = dataiter.next()
                    inputsV = Ytest[i] 
                    labelsV = testlabels[i]
                    inputsV, labelsV = inputsV.to(device), labelsV.to(device)
                    xV = net(inputsV,Kresnet)
                    lossV, SiV = misfit(xV,W,labelsV)
                    accuracyV  = getAccuracy(SiV,labelsV)

#             accuracyV = 0
            print(' %2d      %5d        %5.3f      %5.3f      %5.3f' %
                  (epoch + 1, i + 1, running_loss / p_iter, running_accuracy/p_iter, accuracyV))
            running_loss = 0.0
            running_accuracy = 0.0

print('Finished Training')

Epoch   Iteration   Loss(run)   Acc(run)   Acc(val)
---------------------------------------------------
  1          5        1.386      0.306      0.338
  1          5        1.386      0.327      0.371
  1          5        1.386      0.333      0.417
  1          5        1.386      0.408      0.468
  1          5        1.386      0.432      0.523
  1          5        1.386      0.485      0.584
  1          5        1.386      0.566      0.648
  1          5        1.386      0.680      0.709
  1          5        1.386      0.698      0.757
  1          5        1.385      0.722      0.805
  1          5        1.385      0.779      0.833
Epoch   Iteration   Loss(run)   Acc(run)   Acc(val)
---------------------------------------------------
  2          5        1.385      0.755      0.855
  2          5        1.385      0.821      0.869
  2          5        1.385      0.824      0.887
  2          5        1.385      0.858      0.902
  2          5        1.384      0.864    

In [33]:
# Predict
x = net(Ytest[0],Kresnet)
_,pred = misfit(x,W,testlabels[0])

In [42]:
np.argmax(pred.detach(),axis=1)==testlabels[0]

tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1, 